In [1]:
# !python3 -m venv env
# !source env/bin/activate
# !python3 -m pip install pandas numpy openpyxl

In [2]:
import pandas as pd
import numpy as np

## Read of all tables

In [3]:
data_folder = 'tables'
all_files = os.listdir(data_folder)    
tables = list(filter(lambda f: f.endswith('.xlsx'), all_files))

for table in tables:
    str = table[:-5]
    locals()[str] = pd.read_excel(f'{data_folder}/{table}')
    
print(tables)

['country.xlsx', 'order_status.xlsx', 'order.xlsx', 'vat.xlsx', 'review.xlsx', 'product.xlsx', 'payment_status.xlsx', 'customer.xlsx', 'payment.xlsx', 'category.xlsx', 'address.xlsx', 'order_item.xlsx']


### Task F.1
List all the customer’s names, dates, and products or services used/booked/rented/bought by
these customers in a range of two dates.

In [4]:
suc_orders = order[(order['Date'] >= '2020-01-20') & (order['Date'] <= '2022-02-01') & (order['Status_ID'] == 4)]
suc_orders = suc_orders.merge(customer[['ID', 'First_Name', 'Last_Name']], how = 'left', left_on = ['CUSTOMER_ID'], right_on = ['ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['ID', 'Date', 'First_Name', 'Last_Name', 'CUSTOMER_ID']]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['ID', 'Date', 'First_Name', 'Last_Name', 'CUSTOMER_ID', 'Product_ID']]
suc_orders = suc_orders.merge(product[['ID', 'NAME']], how = 'left', left_on = ['Product_ID'], right_on = ['ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID', 'NAME' : 'PRODUCT_NAME'})
suc_orders['FULL_NAME'] = suc_orders['First_Name'] + ' ' + suc_orders['Last_Name']
suc_orders = suc_orders[['Date', 'FULL_NAME', 'PRODUCT_NAME']]
suc_orders.sort_values(by = ['Date', 'FULL_NAME'], inplace = True)
suc_orders = suc_orders.reset_index(drop = True)
suc_orders


,Date,FULL_NAME,PRODUCT_NAME
0,2020-01-31,Oliver Castro,MacBook Air
1,2020-01-31,Oliver Castro,Samsung A700 series monitor
2,2020-01-31,Oliver Castro,Xiaomi 12T PRO
3,2020-02-06,Jack Oliveira,MacBook Air
4,2020-02-06,Jack Oliveira,BaByliss PRO Titanium Straightener
...,...,...,...
221,2022-01-18,Jack Silva,SonyEricsson K800
222,2022-01-18,Jack Silva,Samsung M7 series monitor
223,2022-01-27,Maria Motta,Sonos Beam (Gen 2)
224,2022-01-31,Anna Motta,Apple watch 8th Gen


### Task F.2
List the best three customers/products/services/places (you are free to define the criteria for
what means “best”)

In [5]:
# Best Customers
suc_orders = order[order['Status_ID'] == 4]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['ID', 'Date', 'CUSTOMER_ID', 'Product_ID', 'Amount']]
suc_orders = suc_orders.merge(product[['ID', 'NAME', 'PRICE']], how = 'left', left_on = ['Product_ID'], right_on = ['ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID', 'NAME' : 'PRODUCT_NAME'})
suc_orders['SPEND'] = suc_orders['Amount'] * suc_orders['PRICE']
suc_orders = suc_orders[['CUSTOMER_ID', 'SPEND']]
suc_orders = suc_orders.groupby(by = ['CUSTOMER_ID']).sum().reset_index()
suc_orders = suc_orders.merge(customer[['ID', 'First_Name', 'Last_Name']], how = 'left', left_on = ['CUSTOMER_ID'], right_on = ['ID'])
suc_orders['FULL_NAME'] = suc_orders['First_Name'] + ' ' + suc_orders['Last_Name']
suc_orders = suc_orders[['FULL_NAME', 'SPEND']].sort_values(by = ['SPEND'], ascending = False)
suc_orders.head(3)

,FULL_NAME,SPEND
4,Ricardo Brown,21463.94
3,Robert Schmidt,13306.98
31,Maria Smith,12441.99


In [6]:
# Best Cities
suc_orders = order[order['Status_ID'] == 4]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['ID', 'Date', 'Address_ID', 'Product_ID', 'Amount']]
suc_orders = suc_orders.merge(product[['ID', 'NAME', 'PRICE']], how = 'left', left_on = ['Product_ID'], right_on = ['ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID', 'NAME' : 'PRODUCT_NAME'})
suc_orders['SPEND'] = suc_orders['Amount'] * suc_orders['PRICE']
suc_orders = suc_orders[['Address_ID', 'SPEND']]
suc_orders = suc_orders.merge(address[['ID', 'Country_ID', 'City']], how = 'left', left_on = ['Address_ID'], right_on = ['ID'])
suc_orders = suc_orders[['Country_ID', 'City', 'SPEND']]
suc_orders = suc_orders.merge(country, how = 'left', left_on = ['Country_ID'], right_on = ['ID'])
suc_orders = suc_orders[['Name', 'City', 'SPEND']]
suc_orders.rename(columns = {'Name' : 'Country_NAME'}, inplace = True)
suc_orders = suc_orders.groupby(by = ['Country_NAME', 'City']).sum().reset_index()
suc_orders = suc_orders.sort_values(by = ['SPEND'], ascending = False)
suc_orders.head(3)

,Country_NAME,City,SPEND
9,Italy,Rome,47662.74
3,France,Marseille,23729.91
0,Canada,London,21886.85


In [7]:
# Best Products
suc_orders = order[order['Status_ID'] == 4]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['ID', 'Date', 'Product_ID', 'Amount']]
suc_orders = suc_orders.merge(product[['ID', 'NAME', 'PRICE']], how = 'left', left_on = ['Product_ID'], right_on = ['ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID', 'NAME' : 'PRODUCT_NAME'})
suc_orders['SPEND'] = suc_orders['Amount'] * suc_orders['PRICE']
suc_orders = suc_orders[['Product_ID', 'PRODUCT_NAME', 'SPEND']]
suc_orders = suc_orders.groupby(by = ['Product_ID', 'PRODUCT_NAME']).sum().reset_index()
suc_orders = suc_orders.sort_values(by = ['SPEND'], ascending = False)
suc_orders.head(3)

,Product_ID,PRODUCT_NAME,SPEND
0,1,iPhone 14 PRO,26622.0
28,29,MacBook Pro,20990.0
37,38,iPad Pro (Latest Model),15980.0


### Task F.3
Get the average amount of sales/bookings/rents/deliveries for a period that involves 2 or more
years, as in the following example.

In [8]:
suc_orders = order[(order['Date'] > '2020-01-01') & (order['Date'] < '2022-03-10') & (order['Status_ID'] == 4)]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['ID', 'Date', 'Product_ID', 'Amount']]
suc_orders = suc_orders.merge(product[['ID', 'NAME', 'PRICE']], how = 'left', left_on = ['Product_ID'], right_on = ['ID'])
suc_orders = suc_orders.rename(columns = {'ID_x': 'ID', 'NAME' : 'PRODUCT_NAME'})
suc_orders['SPEND'] = suc_orders['Amount'] * suc_orders['PRICE']
suc_orders['year'] = pd.to_datetime(suc_orders['Date']).dt.year
suc_orders['month'] = pd.to_datetime(suc_orders['Date']).dt.month
suc_orders = suc_orders[['year', 'month', 'SPEND']]

total = suc_orders['SPEND'].sum()
yearly_avg = total / (suc_orders['year'].nunique() - 1 )
monthly_avg = total / (len(suc_orders[['year', 'month']].drop_duplicates()) + 1)

pd.DataFrame(data = {
    'PeriodOfSales' : ['01/2020 - 03/2022'],
    'TotalSales (euros)' : total,
    'YearlyAverage (of the given period)' : yearly_avg,
    'MonthlyAverage (of the given period)' : monthly_avg
})




,PeriodOfSales,TotalSales (euros),YearlyAverage (of the given period),MonthlyAverage (of the given period)
0,01/2020 - 03/2022,219950.6,109975.3,8146.318519


### Task F.4
Get the total sales/bookings/rents/deliveries by geographical location (city/country).

In [9]:
suc_orders = order[order['Status_ID'] == 4]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])

suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['Address_ID', 'Amount', 'Order_ID']]

suc_orders = suc_orders.merge(address[['ID', 'Country_ID', 'City']], how = 'right', left_on = ['Address_ID'], right_on = ['ID'])
suc_orders = suc_orders.merge(country, how = 'right', left_on = ['Country_ID'], right_on = ['ID'])
suc_orders.drop(columns = ['Address_ID', 'ID_x', 'Country_ID', 'ID_y'], inplace = True)

num_of_orders = suc_orders.groupby(['Name','City'])['Order_ID'].count().reset_index()
num_of_sold_items = suc_orders.groupby(['Name','City'])['Amount'].sum().reset_index()
res = num_of_orders.merge(num_of_sold_items, how = 'left', on = ['City', 'Name'])
res = res.rename(columns = {'Name' : 'Country_NAME', 'Order_ID' : 'N_Orders', 'Amount' : 'N_items_sold'})
res['N_items_sold'] = res['N_items_sold'].astype(int)
res.sort_values(by = ['Country_NAME', 'City'], inplace = True)
res

,Country_NAME,City,N_Orders,N_items_sold
0,Canada,London,18,28
1,Canada,Ontario,16,25
2,Canada,Toronto,0,0
3,France,Lyon,7,10
4,France,Marseille,21,30
5,France,Paris,10,13
6,Germany,Berlin,5,7
7,Germany,Frankfurt,5,8
8,Germany,Hamburg,8,11
9,Germany,Munich,12,22


### Task F.5
List all the locations where products/services were sold, and the product has customer’s ratings

In [10]:
suc_orders = order[order['Status_ID'] == 4]
suc_orders = suc_orders.merge(order_item, how = 'left', left_on = ['ID'], right_on = ['Order_ID'])

suc_orders = suc_orders.rename(columns = {'ID_x': 'ID'})
suc_orders = suc_orders[['Address_ID', 'Product_ID', 'ID_y']]
suc_orders = suc_orders.rename(columns = {'ID_y' : 'Item_id'})

suc_orders = suc_orders.merge(address[['ID', 'Country_ID', 'City']], how = 'left', left_on = ['Address_ID'], right_on = ['ID'])
suc_orders = suc_orders.merge(country, how = 'left', left_on = ['Country_ID'], right_on = ['ID'])
suc_orders = suc_orders[['Name', 'City', 'Product_ID', 'Item_id']]
suc_orders = suc_orders.rename(columns = {'Name' : 'Country_NAME'})
suc_orders = suc_orders.merge(review[['order_item_id', 'Review_Score']], how = 'left', left_on = ['Item_id'], right_on = ['order_item_id'])
suc_orders = suc_orders[['Country_NAME', 'City', 'Product_ID', 'Review_Score']]
suc_orders['tmp'] = 1
tmp_1 = suc_orders.groupby(by = ['Country_NAME','City','Product_ID'])['tmp'].count().reset_index()
tmp_2 = suc_orders.groupby(by = ['Country_NAME','City','Product_ID'])['Review_Score'].mean().reset_index()
res = tmp_1.merge(tmp_2, how = 'left', on = ['Country_NAME', 'City', 'Product_ID'])
res.rename(columns = {'tmp' : 'N_sales', 'Review_Score' : 'AVG_Rating'}, inplace = True)
res = res.merge(product[['ID', 'NAME']], how = 'left', left_on = ['Product_ID'], right_on = ['ID'])
res = res.rename(columns = {'NAME' : 'Product_name'})
res = res[['Country_NAME', 'City', 'Product_ID', 'Product_name', 'N_sales', 'AVG_Rating']]
res

,Country_NAME,City,Product_ID,Product_name,N_sales,AVG_Rating
0,Canada,London,1,iPhone 14 PRO,1,4.0
1,Canada,London,6,iPhone 14 Plus,1,5.0
2,Canada,London,7,iPhone 14 Plus,1,NaN
3,Canada,London,10,Samsung Galaxy S22 ULTRA,1,NaN
4,Canada,London,15,SonyEricsson W200i,1,5.0
...,...,...,...,...,...,...
244,USA,New York,49,Yandex Alice Station,1,5.0
245,USA,New York,51,Apple Pencil Gen 2,1,5.0
246,USA,San Francisco,13,Nokia 3310,1,NaN
247,USA,San Francisco,29,MacBook Pro,1,NaN


In [11]:
res[res['N_sales'] > 1]

,Country_NAME,City,Product_ID,Product_name,N_sales,AVG_Rating
12,Canada,London,40,Samsung Galay Tab s8 Ultra,2,4.0
18,Canada,Ontario,5,iPhone 13,2,5.0
19,Canada,Ontario,7,iPhone 14 Plus,2,4.0
35,France,Lyon,38,iPad Pro (Latest Model),2,4.0
40,France,Marseille,12,Xiaomi 12T PRO,2,5.0
49,France,Marseille,35,LG Ultrafine monitor,2,4.0
54,France,Marseille,54,Apple Magic Mouse,2,NaN
76,Germany,Hamburg,9,iPhone 14 PRO max,2,5.0
82,Germany,Munich,1,iPhone 14 PRO,2,4.0
85,Germany,Munich,19,Apple watch ULTRA,2,5.0


In [12]:
category


,ID,Name,Parent_Category_ID
0,1,SmartPhones,14.0
1,2,CellPhones,14.0
2,3,SmartWatches,15.0
3,4,Scales,16.0
4,5,Face & Body,16.0
5,6,Hair Care,16.0
6,7,Laptops,17.0
7,8,Desktop Computers,17.0
8,9,Monitors,17.0
9,10,Tablets,NaN


In [13]:
review

,ID,Review_Date,order_item_id,Review_Text,Review_Score
0,1,2020-02-06,1,My kid loves it,5
1,2,2020-02-06,2,Amazingly fast shipped!!!,5
2,3,2020-02-06,3,"Wow, it was the best offer",5
3,4,2020-02-12,4,My kid loves it,5
4,5,2020-02-12,5,"Wow, it was the best offer",5
...,...,...,...,...,...
206,207,2022-10-19,360,The color is not so good,4
207,208,2022-10-19,361,"Oh, it is exactly what i needed. Thank you!",5
208,209,2022-10-27,366,"Oh, it is exactly what i needed. Thank you!",5
209,210,2022-10-30,368,"Outstanding quality, as always",5
